**Restaurant Price Predictor**

This projects helps to solve the biggest ever problem whenever we go outside. Especailly the foodies out there. This projects helps to choose the perfect restaurant for you with your likes and also avoid hurting your wallets.



---



PROJECT OVERVIEW

This projects helps you to find the perfect restaurants keeping your needs and most importantly price/budget in consideration and find a perfect place to eat.



---



DATASET OVERVIEW



**INFORMATION**

Dineout is a table booking platform helping customers to do table booking in their favourite restaurants for free and help them get great discounts.The website is well known for displaying user ratings for restaurants, hotels, b&b, touristic attractions, and other places, with a total word count of all reviews is more than 10 million.The Dineout dataset includes thousands of restaurants with attributes such as location data, average rating, number of reviews, cuisine types, etc.The dataset combines the restaurants from the main Indian cities.


**CONTENTS**
*  Name :- Name of the Restaurant   
*  Location :-  Location of the restaurant
*  Locality :-  Locality/Exact address of location
*  City :- City of restaurant
*  Cuisine :- Cuisine served in restaurant
*  Ratings :- Ratings recieved for restaurant
*  Voting :- Votes recieved by toal people for rating of restaurant
*  Cost :- Budget/Cost required.















---



IMPORTING DEPENDECIES

In [54]:
!pip install -q feature_engine autoviz dataprep 2>/dev/null

In [55]:
!pip install catboost


In [56]:
#DataFrame and Numpy Package
import pandas as pd
import numpy as np
#Plotting packages
import seaborn as sns
import matplotlib.pyplot as plt
#Scaling and missing values replacing packages
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
#Feature selection packages
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from feature_engine.encoding import RareLabelEncoder

#Model packages
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#Model Evaluation Packages
from sklearn import tree,metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score,precision_score,recall_score
from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, auc, classification_report
from catboost import Pool, CatBoostRegressor

from sklearn.feature_extraction.text import CountVectorizer
from catboost import Pool, CatBoostRegressor




---



IMPORTING DATA AND ANALYSIS

In [57]:
data_filename = '/content/sample_data/restaurants.csv'

In [58]:
import pandas as pd

In [59]:
df = pd.read_csv(data_filename)

In [60]:
df

,Name,Location,Locality,City,Cuisine,Rating,Votes,Cost
0,Local,"Scindia House,Connaught Place, Central Delhi",Central Delhi,Delhi,"North Indian, Finger Food, Continental",4.1,2415,2000
1,The G.T. ROAD,"M-Block,Connaught Place, Central Delhi",Central Delhi,Delhi,North Indian,4.3,2363,1500
2,Tamasha,"Connaught Place, Central Delhi",Central Delhi,Delhi,"Finger Food, North Indian, Italian, Continental, Asian",4.2,5016,2000
3,The Junkyard Cafe,"Connaught Place, Central Delhi",Central Delhi,Delhi,"North Indian, Mediterranean, Asian, Italian, Oriental",4.2,2821,1800
4,Chili's American Grill and Bar,"M-Block,Connaught Place, Central Delhi",Central Delhi,Delhi,"Mexican, American, Italian",4.4,1094,2000
...,...,...,...,...,...,...,...,...
6588,Shree Rathnam,"Wave Mall,BRS Nagar, West Ludhiana",West Ludhiana,Ludhiana,South Indian,4.1,20,600
6589,Cafe Delish,"Radisson Blu,Rajguru Nagar, West Ludhiana",West Ludhiana,Ludhiana,"Multi-Cuisine, North Indian, Continental, Asian, Health Food",4.4,48,3100
6590,3 K Resorts,"BRS Nagar, West Ludhiana",West Ludhiana,Ludhiana,North Indian,1.0,1,1100
6591,Dovka Bar,"Nirvana Hotel Banquets Club,PAU, Central Ludhiana",Central Ludhiana,Ludhiana,"North Indian, Chinese, Finger Food",4.3,3,1500


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6593 entries, 0 to 6592
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      6593 non-null   object 
 1   Location  6593 non-null   object 
 2   Locality  6593 non-null   object 
 3   City      6593 non-null   object 
 4   Cuisine   6593 non-null   object 
 5   Rating    6593 non-null   float64
 6   Votes     6593 non-null   int64  
 7   Cost      6593 non-null   int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 412.2+ KB


In [62]:
df.describe()

,Rating,Votes,Cost
count,6593.000000,6593.000000,6593.000000
mean,4.088200,119.420143,1102.798271
std,0.670031,261.849704,716.935212
min,1.000000,1.000000,100.000000
25%,3.900000,6.000000,500.000000
50%,4.200000,31.000000,900.000000
75%,4.400000,115.000000,1500.000000
max,5.000000,5016.000000,8000.000000


To check number of numerical columns in out dataset

In [63]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_df = df.select_dtypes(include=numerics)
len(numeric_df.columns)

3

To check wether there are missing data in our dataset and if data is missig we have to find the percentage of data missing in the each column

In [64]:
missing_percentages = df.isna().sum().sort_values(ascending=False) / len(df)
missing_percentages

Name        0.0
Location    0.0
Locality    0.0
City        0.0
Cuisine     0.0
Rating      0.0
Votes       0.0
Cost        0.0
dtype: float64

As we can see there isnt any missing data in our dataset.



---



EXPLORATORY DATA ANALYSIS

In [65]:
item0 = df.shape[0]  # Stores the initial number of rows in the DataFrame
df = df.drop_duplicates()  # Removes duplicate rows from the DataFrame
item1 = df.shape[0]  # Stores the number of rows after removing duplicates
print(f"There are {item0-item1} duplicates found in the dataset")  # Prints the number of duplicates that were removed

# Select only records with positive prices
df = df[df['Cost']>0]

# Normalize locations
df['Location'] = df['Location'].apply(lambda x: ','.join(x.split(',')[-2:]))


# Select only specific columns of interest
selected_cols = ['Cost', 'Location', 'Cuisine', 'Rating', 'Votes']
df = df[selected_cols]

print(df.shape)  # Prints the dimensions (rows and columns) of the filtered DataFrame
df.sample(5).T  # Displays a random sample of 5 rows transposed for better visibility


There are 3 duplicates found in the dataset
(6590, 5)


,1758,2956,516,5640,1944
Cost,500,400,1000,2500,500
Location,"Mira Road, North Western","Nungambakkam, Central Chennai","Punjabi Bagh, West Delhi","Gopalbari, North Jaipur","High Grounds, Central Bangalore"
Cuisine,"North Indian, Chinese, Fast Food",Desserts,"Mughlai, North Indian","North Indian, Italian","Desserts, Ice Cream, Fast Food"
Rating,4.4,4.3,4.2,4.1,4.6
Votes,4,8,25,241,3


In [66]:
df.columns

Index(['Cost', 'Location', 'Cuisine', 'Rating', 'Votes'], dtype='object')

In [68]:
df.Cuisine

0                               North Indian,  Finger Food,  Continental
1                                                           North Indian
2             Finger Food,  North Indian,  Italian,  Continental,  Asian
3             North Indian,  Mediterranean,  Asian,  Italian,  Oriental 
4                                           Mexican,  American,  Italian
                                      ...                               
6588                                                        South Indian
6589    Multi-Cuisine,  North Indian,  Continental,  Asian,  Health Food
6590                                                        North Indian
6591                                North Indian,  Chinese,  Finger Food
6592      Multi-Cuisine,  North Indian,  Italian,  Chinese,  Continental
Name: Cuisine, Length: 6590, dtype: object

In [69]:
cities = df.Cuisine.unique()
len(cities)

2096

In [70]:
# Import the AutoViz_Class
# This class is used for automated exploratory data analysis and visualization.

AV = AutoViz_Class()
from autoviz.AutoViz_Class import AutoViz_Class
%matplotlib inline
AV = AutoViz_Class()

# Initialize variables
filename = ""  # Specify the filename of the dataset (empty in this case)
target_variable = 'Cost'  # Specify the target variable for analysis
custom_plot_dir = "custom_plot_directory"  # Specify the directory to save custom plots

# Perform automated EDA using the AutoViz library
# The following parameters are used:
# - filename: Empty in this case as the data is provided directly as 'df'
# - sep: Delimiter used in the data (comma in this case)
# - depVar: Target variable for analysis ('rating' in this case)
# - dfte: DataFrame to be analyzed ('df' is assumed to be defined earlier)
# - header: Indicates that the first row contains column names (0 for True)
# - verbose: Verbosity level (1 for verbose output)
# - lowess: Smoothing using Lowess algorithm (False for no smoothing)
# - chart_format: Format in which charts will be generated (HTML format in this case)# - chart_format: Format in which charts will be generated (HTML format in this case)
# - max_rows_analyzed: Maximum number of rows to analyze (up to 10,000 rows)
# - max_cols_analyzed: Maximum number of columns to analyze (up to 50 columns)
# - save_plot_dir: Directory to save the generated plots ('custom_plot_directory' in this case)
dft = AV.AutoViz(
    filename,
    sep=",",
    depVar=target_variable,
    dfte=df,
    header=0,
    verbose=1,
    lowess=False,
    chart_format="html",
    max_rows_analyzed=min([df.shape[0], 10**4]),
    max_cols_analyzed=min([df.shape[1], 50]),
    save_plot_dir=custom_plot_dir
)

Shape of your Data Set loaded: (6590, 5)
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
Classifying variables in data set...
    4 Predictors classified...
        No variables removed since no ID or low-information variables found in data set
Since Number of Rows in data 6590 exceeds maximum, randomly sampling 6590 rows for EDA...

################ Regression problem #####################
Saving scatterplots in HTML format
Saving distplots_cats in HTML format
Saving distplots_nums in HTML format
Saving kde_plots in HTML format
Saving violinplots in HTML format
Saving cat_var_plots in HTML format
                                             

Time to run AutoViz (in seconds) = 3


In [71]:
# Import the necessary library for displaying HTML content
from IPython.core.display import display, HTML

# Import the pathlib library to work with file paths
from pathlib import Path

# Initialize an empty list to store file names
file_names = []

# Use pathlib to iterate through HTML files in a specific directory
for file in Path(f'/content/custom_plot_directory/Cost').glob('*.html'):

    # Extract the filename from the full path and add it to the list
    filename = str(file).split('/')[-1]
    file_names.append(filename)

# Iterate through the list of file names and display each HTML file
for file_name in file_names:

    # Construct the full file path for each HTML file
    file_path = f'/content/custom_plot_directory/Cost/{file_name}'

    # Open the HTML file for reading
    with open(file_path, 'r') as file:

       # Read the content of the HTML file
        html_content = file.read()

        # Display the HTML content using IPython
        display(HTML(html_content))


In [72]:
main_label = 'Cost'

# vectorize columns
def vectorize_column(df, col_name, min_df=20):
    ll = df[col_name].fillna('none').str.split(', ').to_list()
    ll = [[j.rstrip(', ').strip(' ').replace('(', '_').replace(')', '_').replace('/', '_').replace('.', '_').replace('+', '_').replace(',', '_').replace('\'', '_').replace(' ', '_').replace('.', '_').replace('&', '_').replace('-', '_').replace('!', '_') for j in i] for i in ll]
    ll1 = []
    for item in ll:
        if item != ['none']:
            ttt = ' '.join(item)
        else:
            ttt = 'none'
        ll1.append(ttt)
    vectorizer = CountVectorizer(max_features=60, min_df=min_df, lowercase=False)
    vectorizer.fit(ll1)
    voc = vectorizer.vocabulary_
    voc_inv = {v: col_name+'_'+k for k, v in voc.items()}
    vector = vectorizer.transform(ll1)
    tt = pd.DataFrame(vector.toarray())
    tt = tt.rename(columns=voc_inv)
    df = pd.concat([df.reset_index(drop=True),tt.reset_index(drop=True)], axis=1).drop([col_name], axis=1)
    return df

for col in ['Cuisine']:
    df = vectorize_column(df, col_name=col, min_df=20)


# log10_transform 'Votes' and group to larger bins
def log10_group_bin(x):
    try:
        return str(round(1/5*round(5*np.log10(1+x)) ,1) )
    except:
        return 'None'
for col in ['Votes']:
    df[f'log10_{col}'] = df[col].apply(log10_group_bin)
    df = df.drop([col], axis=1)

df['Rating'] = df['Rating'].fillna('None').astype(str)

# Set up a rare label encoder for selected columns.
for col in ['Location']:
    df[col] = df[col].fillna('None')
    encoder = RareLabelEncoder(n_categories=1, max_n_categories=100, replace_with='Other', tol=20.0 / df.shape[0])
    df[col] = encoder.fit_transform(df[[col]])

print(df.shape)  # Print the shape of the resulting DataFrame.
df.sample(10).T  # Display a sample of 10 rows, transposed for easier readability.


(6590, 57)


,3349,5477,1750,3533,3825,6380,2662,5116,3291,6321
Cost,1500,900,1800,400,600,700,2000,500,2000,600
Location,Other,Other,Other,Other,Other,Other,Other,Other,"Banjara Hills, Central East Hyderabad",Other
Rating,3.8,4.5,4.3,3.0,3.2,4.5,4.2,4.2,3.9,4.4
Cuisine_American,0,0,0,0,0,0,0,0,0,0
Cuisine_Andhra,0,0,0,0,0,0,0,0,0,0
Cuisine_Arabian,0,0,0,0,0,0,0,0,0,0
Cuisine_Asian,0,0,0,0,0,0,0,0,1,0
Cuisine_Bakery_and_Confectionary,0,0,0,0,0,0,0,0,0,0
Cuisine_Barbecue,0,0,0,0,0,0,0,0,0,0
Cuisine_Bengali,0,0,0,0,0,0,0,0,0,0


In [74]:
from sklearn.model_selection import train_test_split

# Initialize data
# Extract the values of the 'main_label' column and reshape it into a 1D array as 'y'
y = df[main_label].values.reshape(-1,)

# Create the feature matrix 'X' by dropping the 'main_label' column from the DataFrame 'df'
X = df.drop([main_label], axis=1)

# Identify categorical columns in the DataFrame 'df'
# These columns contain non-numeric data
cat_cols = df.select_dtypes(include=['object']).columns

# Create a list of indices for categorical columns in the feature matrix 'X'
cat_cols_idx = [list(X.columns).index(c) for c in cat_cols]

# Split the data into training and testing sets
# - 'X_train' and 'y_train' will contain the training features and labels, respectively
# - 'X_test' and 'y_test' will contain the testing features and labels, respectively
# The split is done with a 50% test size, a random seed of 0, and stratification based on the 'Price' column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0, stratify=df[['Location']])

# Print the dimensions of the training and testing sets
# This provides insight into the sizes of the datasets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3295, 56) (3295, 56) (3295,) (3295,)


In [76]:
%%time

# Initialize the training and testing data pools using CatBoost's Pool class
train_pool = Pool(X_train,
                  y_train,
                  cat_features=cat_cols_idx)  # Create a training data pool with categorical features
test_pool = Pool(X_test,
                 y_test,
                 cat_features=cat_cols_idx)  # Create a testing data pool with categorical features

# Specify the training parameters for the CatBoostRegressor model
model = CatBoostRegressor(iterations=1000,    # Number of boosting iterations
                          depth=5,           # Maximum depth of trees in the ensemble
                          verbose=0,         # Set verbosity level to 0 (no output during training)
                          learning_rate=0.08,  # Learning rate for gradient boosting
                          early_stopping_rounds=10, # Early stopping rounds
                          loss_function='RMSE')  # Loss function to optimize (Root Mean Squared Error)

# Train the CatBoostRegressor model on the training data
model.fit(train_pool, eval_set=test_pool)

# Make predictions using the trained model on both the training and testing data
y_train_pred = model.predict(train_pool)  # Predictions on the training data
y_test_pred = model.predict(test_pool)    # Predictions on the testing data
from sklearn.metrics import mean_squared_error

# Calculate and print the Root Mean Squared Error (RMSE) scores for training and testing data
rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)  # RMSE for training data
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)     # RMSE for testing data

# Print the rounded RMSE scores
print(f"RMSE score for train {round(rmse_train)} INR, and for test {round(rmse_test)} INR")

RMSE score for train 474 INR, and for test 498 INR
CPU times: user 2.66 s, sys: 340 ms, total: 3 s
Wall time: 3.8 s


In [77]:
# Calculate the baseline RMSE (Root Mean Squared Error) scores for the training and test datasets.

# For the training dataset:

# Calculate the RMSE by comparing the actual target values (y_train) with the predicted values,
# where the predicted values are the mean of the training target values repeated for each data sample.
rmse_bs_train = mean_squared_error(y_train, [np.mean(y_train)]*len(y_train), squared=False)

# For the test dataset:

# Calculate the RMSE by comparing the actual target values (y_test) with the predicted values,
# where the predicted values are the mean of the training target values repeated for each test data sample.
rmse_bs_test = mean_squared_error(y_test, [np.mean(y_train)]*len(y_test), squared=False)

# Print the rounded baseline RMSE scores for both the training and test datasets.
print(f"RMSE baseline score for train {round(rmse_bs_train)} INR, and for test {round(rmse_bs_test)} INR")

RMSE baseline score for train 725 INR, and for test 708 INR


SAVING MODEL

In [82]:
import pickle

In [83]:
with open('model.pickle', 'wb') as file:
    pickle.dump(model, file)


In [89]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [91]:
%cd /content/catboost_info

/content/catboost_info


In [92]:
!zip -r CATBOOST.zip /content/catboost_info

  adding: content/catboost_info/ (stored 0%)
  adding: content/catboost_info/time_left.tsv (deflated 51%)
  adding: content/catboost_info/test_error.tsv (deflated 55%)
  adding: content/catboost_info/catboost_training.json (deflated 74%)
  adding: content/catboost_info/learn/ (stored 0%)
  adding: content/catboost_info/learn/events.out.tfevents (deflated 75%)
  adding: content/catboost_info/test/ (stored 0%)
  adding: content/catboost_info/test/events.out.tfevents (deflated 75%)
  adding: content/catboost_info/learn_error.tsv (deflated 53%)
  adding: content/catboost_info/tmp/ (stored 0%)


In [93]:
from google.colab import files
files.download('CATBOOST.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [94]:
%cd /content/custom_plot_directory
!zip -r CUSTOM_PLOT_DIRECTORY.zip /content/custom_plot_directory
from google.colab import files
files.download('CUSTOM_PLOT_DIRECTORY.zip')

/content/custom_plot_directory
  adding: content/custom_plot_directory/ (stored 0%)
  adding: content/custom_plot_directory/Cost/ (stored 0%)
  adding: content/custom_plot_directory/Cost/cat_var_plots.html (deflated 80%)
  adding: content/custom_plot_directory/Cost/distplots_cats.html (deflated 73%)
  adding: content/custom_plot_directory/Cost/kde_plots.html (deflated 76%)
  adding: content/custom_plot_directory/Cost/scatterplots.html (deflated 83%)
  adding: content/custom_plot_directory/Cost/violinplots.html (deflated 86%)
  adding: content/custom_plot_directory/Cost/distplots_nums.html (deflated 64%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [96]:

from google.colab import files
%cd /content/sample_data

files.download('model.pickle')

/content/sample_data


FileNotFoundError: ignored